In [ ]:
import cadquery as cq
import trimesh
import numpy as np
import multiprocessing as mp
from tqdm import tqdm
import pickle
import os

# Load your sloped STEP file
model = cq.importers.importStep("face v1.step")

# Get bounding box
bbox = model.val().BoundingBox()
xmin, xmax = bbox.xmin, bbox.xmax
ymin, ymax = bbox.ymin, bbox.ymax
zmin = bbox.zmin

# Convert CQ model to mesh for raycasting
if os.path.exists('face_mesh.glb'):
    print('we have loaded tmesh')
    tmesh = trimesh.load("face_mesh.glb")
else:
    print('we are computing tmesh')
    vertices, faces = model.val().tessellate(2.0)  # You can tweak this value
    vertices_np = np.array([[v.x, v.y, v.z] for v in vertices])
    faces_np = np.array(faces)
    tmesh = trimesh.Trimesh(vertices=vertices_np, faces=faces_np)
    tmesh.export("face_mesh.glb")

mesh_bytes = pickle.dumps(tmesh)

we have loaded tmesh


In [ ]:
# Parameters
COEFFICIENT = 1.0
spacing = 2 * 4 * COEFFICIENT
cone_radius = 4 * COEFFICIENT # FYI: This cannot be bigger than spacing otherwise we get a kernel crash...
assert cone_radius <= spacing, "cone_radius must be less than or equal to spacing to avoid kernel crash"
cone_depth = 4.0 * COEFFICIENT

RAISED_DEPTH = 0  # cone_depth / 2
connecting_cylinder_radius = 0.75 * COEFFICIENT
connecting_cylinder_depth = cone_depth  # 0.5 * cone_depth
sphere_radius = 1.5 * COEFFICIENT
sphere_depth = connecting_cylinder_depth * 1.5  # (cone_depth + (sphere_radius / 4))


In [ ]:
def build_divot_geometry():
    cone = (
        cq.Workplane("XY")
        .circle(cone_radius)
        .workplane(offset=-cone_depth)
        .circle(0.0001)
        .loft()
        .translate((0, 0, RAISED_DEPTH))
    )

    top_cylinder = (
        cq.Workplane("XY")
        .cylinder(cone_depth * 4, cone_radius)
        .translate((0, 0, cone_depth * 2 + RAISED_DEPTH))
    )

    connecting_cylinder = (
        cq.Workplane("XY")
        .cylinder(connecting_cylinder_depth, connecting_cylinder_radius)
        .translate((0, 0, RAISED_DEPTH - connecting_cylinder_depth))
    )

    sphere = (
        cq.Workplane("XY")
        .sphere(sphere_radius)
        .translate((0, 0, RAISED_DEPTH - sphere_depth))
    )

    return cone.union(top_cylinder).union(connecting_cylinder).union(sphere)


In [ ]:
# Worker function
def create_translated_divot(args):
    x, y, mesh_bytes = args
    try:
        mesh = pickle.loads(mesh_bytes)
        ray_origins = np.array([[x, y, 100]])
        ray_directions = np.array([[0, 0, -1]])
        locations, _, _ = mesh.ray.intersects_location(ray_origins, ray_directions)
        if len(locations) == 0:
            return None
        z_top = locations[0][2]
        divot = build_divot_geometry()
        return divot.translate((x, y, z_top))
    except Exception as e:
        print(f"Failed divot at ({x}, {y}): {e}")
        return None



In [ ]:
from concurrent.futures import ProcessPoolExecutor
import numpy as np
import pickle

def get_z_top(mesh_bytes, x, y):
    mesh = pickle.loads(mesh_bytes)
    ray_origins = np.array([[x, y, 100]])
    ray_directions = np.array([[0, 0, -1]])
    locations, _, _ = mesh.ray.intersects_location(ray_origins, ray_directions)
    return locations[0][2] if len(locations) > 0 else None

def create_translated_divot(args):
    x, y, mesh_bytes = args
    try:
        mesh = pickle.loads(mesh_bytes)
        ray_origins = np.array([[x, y, 100]])
        ray_directions = np.array([[0, 0, -1]])
        locations, _, _ = mesh.ray.intersects_location(ray_origins, ray_directions)
        if len(locations) == 0:
            return None
        z_top = locations[0][2]
        divot = build_divot_geometry()
        return divot.translate((x, y, z_top))
    except Exception as e:
        print(f"Failed divot at ({x}, {y}): {e}")
        return None


In [ ]:
# Build argument list
args = [
    (x, y, mesh_bytes)
    for x in range(int(xmin), int(xmax) + 2, int(spacing))
    for y in range(int(ymin), int(ymax) + 2, int(spacing))
]

divots = cq.Workplane("XY")

with mp.Pool(mp.cpu_count()) as pool:
    for result in tqdm(pool.imap_unordered(create_translated_divot, args), total=len(args)):
        if result is not None:
            divots = divots.union(result)

# Subtract divots from model
modified = model.cut(divots)

# Export result
cq.exporters.export(modified, "divotted_face_v1.step")


  0%|          | 0/260 [00:00<?, ?it/s]Process SpawnPoolWorker-1:
Traceback (most recent call last):
Process SpawnPoolWorker-2:
Traceback (most recent call last):
  File "/opt/miniconda3/envs/ethical_necromancy/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/miniconda3/envs/ethical_necromancy/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/miniconda3/envs/ethical_necromancy/lib/python3.11/multiprocessing/pool.py", line 114, in worker
    task = get()
           ^^^^^
  File "/opt/miniconda3/envs/ethical_necromancy/lib/python3.11/multiprocessing/queues.py", line 367, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'create_translated_divot' on <module '__main__' (built-in)>
  File "/opt/miniconda3/envs/ethical_necromancy/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run

KeyboardInterrupt: 